In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unique-bvbinfo-tournament-id/unique_tournid.csv


In [2]:
tourn_id_df = pd.read_csv("/kaggle/input/unique-bvbinfo-tournament-id/unique_tournid.csv")
tourn_id_df.info()                 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   TOURNID  50 non-null     int64
dtypes: int64(1)
memory usage: 528.0 bytes


In [3]:
base_url = "http://www.bvbinfo.com/Tournament.asp?ID="

In [4]:
import requests
from bs4 import BeautifulSoup

def bvbinfo_tourni_scraper(tourn_id):
    url = base_url + str(tourn_id)
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extracting team information from the table
  
    table = soup.find_all('table')
    print("len: ", len(table))
    if len(table) < 5:
        return []
    rows = table[4].find_all('tr')
    header_element = rows[0]
    header_text = header_element.get_text(strip=True, separator="\n").split("\n")
    # Extract category, tournament, location, and start-end date
    category = header_text[0].split("'")[0]  
    tournament = header_text[0].split()[-1]  
    location = header_text[0].split()[-2]  
    startenddate = header_text[1]  

    event = tournament + " " + category
    # Split by space to get month and day range
    month, day_range, year = startenddate.split()[:3]

    # Split day range by '-'
    start_day, end_day = day_range.split('-')

    # Construct tourniStartDate and tourniEndDate
    tourniStartDate = f"{month} {start_day}, {year}"
    tourniEndDate = f"{month} {end_day}, {year}"
    
    misc_info = []
    misc_info =[tourniStartDate, tourniStartDate, location, event, tourn_id]
    data = []
    for cols_element in rows[1:]:
        values = []
        players = []
        if 'Canada' in cols_element.get_text():
            for col in cols_element.find_all('td')[:-4]: # don't need all columns
                a_tag = col.find('a', href=True)
                if a_tag and 'player.asp' in a_tag['href']:
                    value = a_tag['href'].split('=')[-1]
                    players.append(value)
                #else:
                value = col.text.strip()
                values.append(value)
            data.append(values[:3] + players + values[3:] + misc_info)
                 
    return data


In [5]:
columns = ['Finish', 'Player', 'Partner', 'Player_ID', 'Partner_ID', 'Country'
           , 'TourniStartDate', 'TourniEndDate', 'TournLocation', 'Event', 'TOURNID']
scraped_df = pd.DataFrame(columns=columns)

for tournament_id in tourn_id_df['TOURNID']:
    data = bvbinfo_tourni_scraper(tournament_id)
    if data:  # Check if data is not empty
        temp_df = pd.DataFrame(data, columns=columns)
        scraped_df = pd.concat([scraped_df, temp_df], ignore_index=True)

http://www.bvbinfo.com/Tournament.asp?ID=3962
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3968
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3970
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3972
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3992
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3994
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3964
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3960
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3982
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=4147
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=4149
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=4151
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3961
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3967
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3969
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3971
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3991
len:  6
http://www.bvbinfo.com/Tournament.asp?ID=3993
len:  6
http://www.bvbinfo.com/Tourn

In [6]:
scraped_df.to_csv("scraped_tournament_info.csv")